# Emergency department

Many patients admitted to the intensive care unit (ICU) originally present in the emergency department (ED). Information on care provided is not directly a part of MIMIC-III, though there are some pieces of information about patient ED stays which are available. Information available includes: (1) admission information (2) (infrequently) fluids/treatment received.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
from IPython.display import display, HTML # used to print out pretty pandas dataframes
import matplotlib.dates as dates
import matplotlib.lines as mlines

%matplotlib inline
plt.style.use('ggplot') 

# specify user/password/where the database is
sqluser = 'postgres'
sqlpass = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
host = 'localhost'

query_schema = 'SET search_path to ' + schema_name + ';'

# connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=sqlpass, host=host)

In [2]:
# load the admissions table from the local database
query = query_schema + 'select * from admissions'
df = pd.read_sql_query(query,con)

# define a function to only output human readable dates
def pretty_dates(x):
    if 'NaTType' in str(type(x)):
        return ''
    else:
        return x.strftime('%m-%d %H:%M')

df['ed-registered'] = df['edregtime'].apply(pretty_dates)
df['ed-out'] = df['edouttime'].apply(pretty_dates)

df['hosp-admit'] = df['admittime'].apply(pretty_dates)
df['hosp-disch'] = df['dischtime'].apply(pretty_dates)

# look at the top 5 rows for the table
df[['ed-registered','ed-out','admission_location','hosp-admit','hosp-disch']].head()

,ed-registered,ed-out,admission_location,hosp-admit,hosp-disch
0,04-09 10:06,04-09 13:24,EMERGENCY ROOM ADMIT,04-09 12:26,04-10 15:54
1,,,PHYS REFERRAL/NORMAL DELI,09-03 07:15,09-08 19:10
2,,,TRANSFER FROM HOSP/EXTRAM,10-18 19:34,10-25 14:00
3,,,TRANSFER FROM HOSP/EXTRAM,06-06 16:14,06-09 12:48
4,11-02 01:01,11-02 04:27,EMERGENCY ROOM ADMIT,11-02 02:06,11-05 14:55


Above we can see how the emergency department (ED) times in the admissions table work. Let's look at the top row (index 0): the patient is registered in the ED at 10:06, and is recorded as leaving the ED at 13:24. During this time they have been admitted to the hospital (`admittime` of 12:26). While it seems odd to be in the ED and the hospital at the same time, keep in mind that the ED and the hospital are operating on different systems, and `admittime` is an administrative database, not a real-time GPS tracker!

In [3]:
df.groupby('admission_location')[['edregtime','edouttime']].count().sort_values('edregtime', ascending=False)

,edregtime,edouttime
admission_location,,
EMERGENCY ROOM ADMIT,22437,22437
CLINIC REFERRAL/PREMATURE,7932,7932
TRANSFER FROM HOSP/EXTRAM,196,196
TRANSFER FROM SKILLED NUR,192,192
PHYS REFERRAL/NORMAL DELI,103,103
TRANSFER FROM OTHER HEALT,14,14
** INFO NOT AVAILABLE **,2,2
HMO REFERRAL/SICK,1,1
TRSF WITHIN THIS FACILITY,0,0


With the above group, we can see that most patients who go through the ED are direct admissions, but sometimes clinical referrals and transfers also go via the ED.

## Prescriptions

@msjoding asks:

> I see that the PRESCRIPTIONS table includes meds that were ordered outside of ICU stays. Does this table also include any medications that were ordered in the emergency department just prior to the hospital admission?

We can investigate this by cross-referencing hospital admission times with prescriptions. However, since prescriptions are only available with a date, we need to isolate to admissions where the patient was in the ED for at least a calendar day.

In [4]:
# load entire prescriptions table into memory
query = query_schema + 'select * from prescriptions'
pr = pd.read_sql_query(query,con)

In [5]:
# first, isolate to patients in the ed
ed = df.loc[df['edregtime'].notnull(), :]

# now find patients in the ED but not in the hospital
ed_day = ed['edregtime'].dt.floor('d')
hosp_day = ed['admittime'].dt.floor('d')
hosp_hour = ed['admittime'].apply(lambda x: x.hour)

# add a gap
idx = (hosp_day > ed_day) & (hosp_hour > 6)
ed = ed.loc[idx, :]

# look at the top 5 rows for the table
ed[['ed-registered','ed-out','admission_location','hosp-admit','hosp-disch']].head()

,ed-registered,ed-out,admission_location,hosp-admit,hosp-disch
40,12-15 21:10,12-16 22:05,EMERGENCY ROOM ADMIT,12-16 19:48,12-22 16:15
143,05-17 15:27,05-18 11:50,EMERGENCY ROOM ADMIT,05-18 11:06,05-26 14:30
285,03-31 23:28,04-01 08:28,EMERGENCY ROOM ADMIT,04-01 07:11,04-18 15:25
309,04-21 21:40,04-22 09:40,EMERGENCY ROOM ADMIT,04-22 08:44,05-05 02:04
380,04-06 20:11,04-07 23:50,EMERGENCY ROOM ADMIT,04-07 11:59,06-06 14:21


Above, row index 143 looks promising: they were registered in the ED at 15:27 and admitted to the hospital the next day at 11:06.

In [6]:
subject_id = ed.loc[143, 'subject_id']

# how many admissions does this patient have?
idx = df['subject_id']==subject_id
print('{} hospital admissions for subject.'.format(df.loc[idx, 'hadm_id'].count()))
print('Row {} is the first admission.'.format(
    df.loc[idx, :].sort_values('admittime', ascending=True).index[0]))

query = query_schema + 'select * from prescriptions where subject_id = {}'.format(subject_id)
pr = pd.read_sql_query(query,con)

# truncate the admittime to day
admitdate = pd.Timestamp(ed.loc[143, 'admittime'].date())

# look for rows before this day
idx = (pr['subject_id'] == subject_id) & (pr['startdate'] < admitdate)
pr.loc[idx, :].sort_values('startdate')

2 hospital admissions for subject.
Row 143 is the first admission.


,row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route,mimic_id


Given the above, it seems unlikely the prescriptions table captures ED medications.

In [7]:
n = 0
for i, row in ed.iterrows():
    # get the subject ID
    subject_id = row['subject_id']
    
    # skip this row if it is a secondary admission to the hospital
    idx = df['subject_id']==subject_id
    if i != df.loc[idx, :].sort_values('admittime', ascending=True).index[0]:
        continue
        
    # truncate the admittime to day
    admitdate = pd.Timestamp(row['admittime'].date())

    # look for rows before this day but no more than 7 days before
    idx = (pr['subject_id'] == subject_id) & (pr['startdate'] < admitdate)
    print('subject_id {:5d} - {} rows.'.format(subject_id, idx.sum()))
    n+=1
    
    if n>20:
        break

subject_id   135 - 0 rows.
subject_id   453 - 0 rows.
subject_id   384 - 0 rows.
subject_id   407 - 0 rows.
subject_id   209 - 0 rows.
subject_id   878 - 0 rows.
subject_id   653 - 0 rows.
subject_id  1032 - 0 rows.
subject_id  1092 - 0 rows.
subject_id   702 - 0 rows.
subject_id   810 - 0 rows.
subject_id   822 - 0 rows.
subject_id  1245 - 0 rows.
subject_id  1119 - 0 rows.
subject_id  1136 - 0 rows.
subject_id  1541 - 0 rows.
subject_id  1630 - 0 rows.
subject_id  1683 - 0 rows.
subject_id  1262 - 0 rows.
subject_id  1297 - 0 rows.
subject_id  2001 - 0 rows.


It would appear that no emergency department prescriptions exist in the `prescriptions` table!